![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTK4gQ9nhwHHaSXMHpeggWg7twwMCgb877smkRmtkmDeDoGF9Z6&usqp=CAU)

# <font color='PURPLE'>Ciência dos Dados na Prática</font>

# <font color='GREY'> Detecção de Fraude em Cartão de Crédito</font>



![alt text](https://www.credilink.com.br/wp-content/uploads/2016/07/fraudes-cartoes-credito-loja-virtual.gif)

https://www.kaggle.com/mlg-ulb/creditcardfraud

# 1° Qual o Problema de Negócio

* Os conjuntos de dados **contêm transações** feitas por cartões de crédito em setembro de 2013 por titulares de cartões europeus.

* Este conjunto de dados apresenta as transações que ocorreram em dois dias, onde **temos 492 fraudes em 284.807 transações**. O conjunto de dados é altamente desequilibrado, a classe positiva (fraudes) é responsável por 0,172% de todas as transações.

* Ele contém apenas variáveis ​​de entrada numéricas que são o resultado de uma **transformação PCA**. Infelizmente, devido a questões de **confidencialidade**, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Os recursos V1, V2,… V28 são os componentes principais obtidos com o PCA, os únicos recursos que não foram transformados com o PCA são 'Tempo' e 'Quantidade'. 

* O recurso 'Tempo' contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. O recurso 'Amount' é o Amount da transação, esse recurso pode ser usado, por exemplo, para estudos diversos. 

* O recurso **'Classe' é a variável de resposta** e assume o valor 1 em caso de fraude e 0 em caso contrário.



# 2° Análise Exploratória dos Dados

In [4]:
# Bibliotecas utilizadas
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [5]:
# Lendo os Dados
df = pd.read_csv('/content/creditcard.csv')

In [6]:
# Informações dos Atributos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [7]:
# Visualizando algumas linhas
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
# Verificando valores missing
# (verificar linhas duplicadas)
df.isna().sum()

In [9]:
# Eliminando os valores missing
df.dropna(inplace=True) 

In [ ]:
# Dando uma olhada na variável Target
df['Class'].value_counts()
sns.countplot(df['Class'])

In [11]:
# Explorando as correlações com o Target
df.corr()['Class'].sort_values(ascending=False)

Class     1.000000
V11       0.154876
V4        0.133447
V2        0.091289
V21       0.040413
V19       0.034783
V20       0.020090
V8        0.019875
V27       0.017580
V28       0.009536
Amount    0.005632
V26       0.004455
V25       0.003308
V22       0.000805
V23      -0.002685
V15      -0.004223
V13      -0.004570
V24      -0.007221
Time     -0.012323
V6       -0.043643
V5       -0.094974
V9       -0.097733
V1       -0.101347
V18      -0.111485
V7       -0.187257
V3       -0.192961
V16      -0.196539
V10      -0.216883
V12      -0.260593
V14      -0.302544
V17      -0.326481
Name: Class, dtype: float64

In [ ]:
# Estatísticas descritivas das variáveis
df.describe()

# 3° Pré-Processamento dos dados

* Esta etapa normalmente leva mais tempo no ciclo, mas para nossos
propósitos, a maioria dos conjuntos de dados escolhidos nesta série são limpos.

* Os conjuntos de dados do mundo real são bagunçados e incompletos. As escolhas que fazemos nesta etapa para resolver problemas de dados podem afetar as etapas posteriores e o próprio resultado final. 

* Por exemplo, pode ser complicado lidar com dados ausentes quando não sabemos por que eles estão ausentes. 

* É missing mesmo, ou a ausência de Informação tem um significado específico? 

* Também pode ser complicado lidar com outliers se não entendermos o domínio e o contexto do problema o suficiente.



In [13]:
# Separando os dados para treinamento e teste da máquina preditiva
X = df.drop(['Class'], axis=1)
y = df['Class']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=7)

# 4° Criando a Máquina Preditiva de Detecção de Fraude

* Criamos a Máquina Preditiva e ajustamos o modelo aos dados que preparamos para o treinamento.

* Em seguida, procedemos à classificação com os dados que preparamos para avaliação.

* Por último, avaliamos o desempenho do modelo com as principais métricas de classificação.

In [14]:
# Construindo o Modelo
model = RandomForestClassifier()

#from xgboost import XGBClassifier
#model = XGBClassifier()

In [15]:
# Treinando o Modelo
# Demora alguns minutinhos
model = model.fit(X_train, y_train)

In [16]:
# Passando novos dados (dados de teste) Para o modelo
y_predict = model.predict(X_val)

# 5° Avaliando a Máquina Preditiva

In [18]:
# Comparando gabarito e Previsoes_da_Máquina
gabarito = pd.DataFrame({'Gabarito': y_val,'Previsoes_da_Máquina': y_predict})
gabarito

,Gabarito,Previsoes_da_Máquina
86801,0,0
34867,0,0
151239,0,0
122560,0,0
77820,0,0
...,...,...
11519,0,0
21449,0,0
129577,0,0
197268,0,0


In [19]:
# Evaluate model
print('Classification metrics: \n', classification_report(y_val, y_predict))

Classification metrics: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     85288
           1       0.93      0.82      0.87       155

    accuracy                           1.00     85443
   macro avg       0.96      0.91      0.93     85443
weighted avg       1.00      1.00      1.00     85443



### **Observações Importantes**

* Essas são as melhores métricas para avaliar esse modelo? (AUC...?)
* Qual tratamento preciso fazer para lidar com o Target desbalanceado? (SMOTE...)
* Qual a melhor forma de fazer a amostra dos dados? (CV...)
* Preciso Padronizar/Normalizar os Dados? (StandartScaler...)
* Utilizo todas as Variáveis? (Drop "time"...)
* Quais Algoritmos também poderiam ser utilizados?(LGBM, AdaBoost..)
* Como avaliar o Overfiting? 


# Fim

## Valeu!

### #Instagram - Ciencia dos Dados <a href="http://instagram.com/cienciadosdados">instagram.com/cienciadosdados</a>

### #Facebook - Ciencia dos Dados <a href="http://facebook.com/cienciadosdadosbr">facebook.com/cienciadosdadosbr</a>

### #**Telegram** - Scripts e Datasets - Comunidade Telegram <a href="https://t.me/cienciadosdadosraiz">https://t.me/cienciadosdadosraiz</a>

### #YouTube - Mais Aulas como essa no YouTube <a href="https://www.youtube.com/watch?v=IaIc5oHd3II&t=1569s">https://www.youtube.com/watch?v=IaIc5oHd3II&t=1569s</a>

In [ ]:
from IPython.core.display import HTML
HTML('<iframe width="380" height="200" src="https://www.youtube.com/embed/Z0vDDKXzAhA" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')